# **Distribución de créditos por monto**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "pachuca" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_pachuca',
 'df_financiamientos_2020_pachuca',
 'df_financiamientos_2021_pachuca',
 'df_financiamientos_2022_pachuca',
 'df_financiamientos_2023_pachuca',
 'df_financiamientos_2024_pachuca']

In [2]:
df_financiamientos_2022_pachuca = df_financiamientos_2022_pachuca[['id','monto']]
df_financiamientos_2023_pachuca = df_financiamientos_2023_pachuca[['id','monto']]
df_financiamientos_2024_pachuca = df_financiamientos_2024_pachuca[['id','monto']]

## 2022

In [3]:
# Definir los rangos de los monto
segmentos = {
    '1': (0.00, 1000000.00),
    '2': (1000001.00, 2000000.00),
    '3': (2000001.00, 3000000.00),
    '4': (3000001.00, 4000000.00),
    '5': (4000001.00, float('inf'))  # Rango con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2022_pachuca['rango'] = df_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)
df_financiamientos_2022_pachuca.head()


,id,monto,rango
0,1,0.00,1
1,2,1563551.33,2
2,3,1263770.32,2
3,4,3614324.42,4
4,5,252907.18,1


In [4]:
conteo_por_rango = df_financiamientos_2022_pachuca['rango'].value_counts()
df_conteo_por_rango = pd.DataFrame({
    'rango': conteo_por_rango.index,
    'conteo': conteo_por_rango.values
})
# Nuevo registro que queremos agregar
#nuevo_registro = {'rango': 5, 'conteo': 0}
# Agregar el nuevo registro al DataFrame
#df_conteo_por_rango = df_conteo_por_rango.append(nuevo_registro, ignore_index=True)
df_conteo_por_rango

,rango,conteo
0,1,758
1,2,333
2,3,137
3,5,104
4,4,56


In [5]:
df_conteo_por_rango = df_conteo_por_rango.sort_values(by='rango', ascending=True)
df_conteo_por_rango

,rango,conteo
0,1,758
1,2,333
2,3,137
4,4,56
3,5,104


In [6]:
# Convertir la columna 'rango' a cadena de texto (str) antes de realizar el reemplazo
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].astype(str)

# Reemplazar los valores de rango con el texto correspondiente utilizando replace
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].replace({
    #'1': '< 1M',
    #'2': '< 2M',
    #'3': '< 3M',
    #'4': '< 4M',
    #'5': '< 20M'    
    #'1': '$0.00 - $1,000,000.00',
    #'2': '$1,000,001.00 - $2,000,000.00',
    #'3': '$2,000,001.00 - $3,000,000.00',
    #'4': '$3,000,001.00 - $4,000,000.00',
    #'5': '$4,000,001.00 - $20,000,000.00'    
   '1': '$0 - 1 millón',
    '2': '$1 - 2 millones',
    '3': '$2 - 3 millones',
    '4': '$3 - 4 millones',
    '5': '$4 - 20 millones'


})
df_conteo_por_rango

,rango,conteo
0,$0 - 1 millón,758
1,$1 - 2 millones,333
2,$2 - 3 millones,137
4,$3 - 4 millones,56
3,$4 - 20 millones,104


In [7]:
labels = df_conteo_por_rango['rango']
values = df_conteo_por_rango['conteo']
#trimestre = tabla_porcentajes['trimestre']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
#trimestres = tabla_porcentajes.columns
fig.add_trace(go.Bar(
        #name=f'Trimestre {trimestre}', 
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por monto 2022',
    #title_font_size = 20,
    xaxis=dict(title="Monto"),
    yaxis=dict(
        title='Créditos', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)


guardar_grafico_como_html(fig, 'g_bar_creditos_monto_2022',  carpeta='assets/graficas')
fig.show()

## 2023

In [8]:
# Definir los rangos de los monto
segmentos = {
    '1': (0.00, 1000000.00),
    '2': (1000001.00, 2000000.00),
    '3': (2000001.00, 3000000.00),
    '4': (3000001.00, 4000000.00),
    '5': (4000001.00, float('inf'))  # Rango con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2023_pachuca['rango'] = df_financiamientos_2023_pachuca['monto'].apply(asignar_segmento)
df_financiamientos_2023_pachuca.head()


,id,monto,rango
0,1,0.00,1
1,2,486903.48,1
2,3,5702163.94,5
3,4,5284544.47,5
4,5,691154.29,1


In [9]:
conteo_por_rango = df_financiamientos_2023_pachuca['rango'].value_counts()
df_conteo_por_rango = pd.DataFrame({
    'rango': conteo_por_rango.index,
    'conteo': conteo_por_rango.values
})
# Nuevo registro que queremos agregar
#nuevo_registro = {'rango': 5, 'conteo': 0}
# Agregar el nuevo registro al DataFrame
#df_conteo_por_rango = df_conteo_por_rango.append(nuevo_registro, ignore_index=True)
df_conteo_por_rango

,rango,conteo
0,1,790
1,2,341
2,3,118
3,5,97
4,4,53


In [10]:
df_conteo_por_rango = df_conteo_por_rango.sort_values(by='rango', ascending=True)
df_conteo_por_rango

,rango,conteo
0,1,790
1,2,341
2,3,118
4,4,53
3,5,97


In [11]:
# Convertir la columna 'rango' a cadena de texto (str) antes de realizar el reemplazo
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].astype(str)

# Reemplazar los valores de rango con el texto correspondiente utilizando replace
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].replace({
    #'1': '< 1M',
    #'2': '< 2M',
    #'3': '< 3M',
    #'4': '< 4M',
    #'5': '< 20M'    
    #'1': '$0.00 - $1,000,000.00',
    #'2': '$1,000,001.00 - $2,000,000.00',
    #'3': '$2,000,001.00 - $3,000,000.00',
    #'4': '$3,000,001.00 - $4,000,000.00',
    #'5': '$4,000,001.00 - $20,000,000.00'    
'1': '$0 - 1 millón',
'2': '$1 - 2 millones',
'3': '$2 - 3 millones',
'4': '$3 - 4 millones',
'5': '$4 - 20 millones'

})
df_conteo_por_rango

,rango,conteo
0,$0 - 1 millón,790
1,$1 - 2 millones,341
2,$2 - 3 millones,118
4,$3 - 4 millones,53
3,$4 - 20 millones,97


In [12]:
labels = df_conteo_por_rango['rango']
values = df_conteo_por_rango['conteo']
#trimestre = tabla_porcentajes['trimestre']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
#trimestres = tabla_porcentajes.columns
fig.add_trace(go.Bar(
        #name=f'Trimestre {trimestre}', 
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por monto 2022',
    #title_font_size = 20,
    xaxis=dict(title="Monto"),
    yaxis=dict(
        title='Créditos', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)


guardar_grafico_como_html(fig, 'g_bar_creditos_monto_2023',  carpeta='assets/graficas')
fig.show()

## 2024

In [13]:
# Definir los rangos de los monto
segmentos = {
    '1': (0.00, 1000000.00),
    '2': (1000001.00, 2000000.00),
    '3': (2000001.00, 3000000.00),
    '4': (3000001.00, 4000000.00),
    '5': (4000001.00, float('inf'))  # Rango con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2024_pachuca['rango'] = df_financiamientos_2024_pachuca['monto'].apply(asignar_segmento)
df_financiamientos_2024_pachuca.head()


,id,monto,rango
0,1,5479808.78,5
1,2,1715027.31,2
2,3,938335.09,1
3,4,1014831.18,2
4,5,2344630.70,3


In [14]:
conteo_por_rango = df_financiamientos_2024_pachuca['rango'].value_counts()
df_conteo_por_rango = pd.DataFrame({
    'rango': conteo_por_rango.index,
    'conteo': conteo_por_rango.values
})
# Nuevo registro que queremos agregar
#nuevo_registro = {'rango': 5, 'conteo': 0}
# Agregar el nuevo registro al DataFrame
#df_conteo_por_rango = df_conteo_por_rango.append(nuevo_registro, ignore_index=True)
df_conteo_por_rango

,rango,conteo
0,1,424
1,2,191
2,3,76
3,5,59
4,4,42


In [15]:
df_conteo_por_rango = df_conteo_por_rango.sort_values(by='rango', ascending=True)
df_conteo_por_rango

,rango,conteo
0,1,424
1,2,191
2,3,76
4,4,42
3,5,59


In [16]:
# Convertir la columna 'rango' a cadena de texto (str) antes de realizar el reemplazo
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].astype(str)

# Reemplazar los valores de rango con el texto correspondiente utilizando replace
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].replace({
    #'1': '< 1M',
    #'2': '< 2M',
    #'3': '< 3M',
    #'4': '< 4M',
    #'5': '< 20M'    
    #'1': '$0.00 - $1,000,000.00',
    #'2': '$1,000,001.00 - $2,000,000.00',
    #'3': '$2,000,001.00 - $3,000,000.00',
    #'4': '$3,000,001.00 - $4,000,000.00',
    #'5': '$4,000,001.00 - $20,000,000.00'    
    '1': '$0 - 1 millón',
'2': '$1 - 2 millones',
'3': '$2 - 3 millones',
'4': '$3 - 4 millones',
'5': '$4 - 20 millones'

})
df_conteo_por_rango

,rango,conteo
0,$0 - 1 millón,424
1,$1 - 2 millones,191
2,$2 - 3 millones,76
4,$3 - 4 millones,42
3,$4 - 20 millones,59


In [17]:
labels = df_conteo_por_rango['rango']
values = df_conteo_por_rango['conteo']
#trimestre = tabla_porcentajes['trimestre']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
#trimestres = tabla_porcentajes.columns
fig.add_trace(go.Bar(
        #name=f'Trimestre {trimestre}', 
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por monto 2022',
    #title_font_size = 20,
    xaxis=dict(title="Monto"),
    yaxis=dict(
        title='Créditos', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)


guardar_grafico_como_html(fig, 'g_bar_creditos_monto_2024',  carpeta='assets/graficas')
fig.show()